In [1]:
# Auto-Reload Setup für Entwicklung
# Automatisches Neuladen aller Module bei Änderungen
%load_ext autoreload
%autoreload 2

In [2]:
# Imports - werden automatisch neu geladen bei Änderungen
from datetime import datetime
import pandas as pd
from state_provider.state_provider import state_provider

print("✅ Module erfolgreich importiert mit Auto-Reload aktiviert")

✅ Module erfolgreich importiert mit Auto-Reload aktiviert


In [7]:
# Daten laden und parsen
CSV = "data/full_test_export.csv"
df = pd.read_csv(CSV, sep=";")
state = state_provider.parse_data_to_state(df)

print("✅ Daten erfolgreich geparst")

2025-12-07 14:43:19.544 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.545 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.551 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.552 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.545 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:19.551 WARNING streamlit.run

Data time range: 2025-09-10 11:53:00 - 2025-09-30 01:16:00
✅ Daten erfolgreich geparst


In [8]:
# Verify queries after cleanup
print("Testing Vitals Query:")
vitals = state_provider.query_data("vitals")
print(f"Vitals rows: {len(vitals)}")

print("\nTesting Lab Query:")
lab = state_provider.query_data("lab")
print(f"Lab rows: {len(lab)}")

print("\nTesting Device Query:")
ecmo = state_provider.query_data("ecmo")
print(f"ECMO rows: {len(ecmo)}")

2025-12-07 14:43:23.138 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.138 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.145 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 14:43:23.145 WARNING streamlit.run

Testing Vitals Query:
Vitals rows: 3387

Testing Lab Query:
Lab rows: 7490

Testing Device Query:
ECMO rows: 236


In [4]:
# Hilfsfunktion: Alle verfügbaren Methoden anzeigen
def show_available_methods():
    methods = [method for method in dir(state_provider) if not method.startswith('_')]
    print("Verfügbare StateProvider Methoden:")
    for i, method in enumerate(methods, 1):
        print(f"  {i:2}. {method}")
    return methods

# Methoden anzeigen
available_methods = show_available_methods()

Verfügbare StateProvider Methoden:
   1. data_manager
   2. get_device_time_ranges
   3. get_lab_form
   4. get_nearest_ecls_time
   5. get_nearest_impella_time
   6. get_record_id
   7. get_respiration_type
   8. get_respiratory_value
   9. get_selected_time_range
  10. get_selected_view
  11. get_state
  12. get_time_of_mcs
  13. get_time_range
  14. get_value_strategy
  15. get_vasoactive_agents_df
  16. get_vitals_value
  17. has_device_past_24h
  18. has_mcs_records_past_24h
  19. has_parsed_data
  20. parse_data_to_state
  21. query_data
  22. query_manager
  23. reset_state
  24. save_state
  25. set_selected_time_range
  26. update_lab_form_field
  27. update_state


In [5]:
timerange = (df["timestamp"].min(), df["timestamp"].max())
timerange

(Timestamp('2025-09-10 11:53:00'), Timestamp('2025-09-30 01:16:00'))